In [3]:
import arcgis
import json
import time as tm
import numpy as np
import sqlite3
import logging
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path
from arcgis.gis import GIS
from arcgis.geometry import Point, Polyline, Polygon

In [4]:
def find_project_root(search_filename=".git"):
    """
    Find the root directory of the project by looking for a specific file or directory.

    :param search_filename: The filename or directory to look for to identify the project root.
                            Defaults to '.git'.
    :return: The absolute path to the root directory of the project.
    """
    current_path = Path(__file__).resolve()

    for parent in current_path.parents:
        if (parent / search_filename).exists():
            return parent

    raise FileNotFoundError(f"Could not find the project root containing {search_filename}")

In [5]:
logging.basicConfig(filename='line_matching.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')


class ShortestPath:

    @staticmethod
    def dijkstra(graph: list, start: int, end: int) -> tuple:
        """
        Dijkstra's algorithm for finding the shortest path in a graph.
        :param graph: a graph of nodes in adjacency matrix form
        :param start: source
        :param end: target
        :return: list of shortest distances from node 0 to the last node
        """

        # noinspection PyTypeChecker
        def reconstruct(distances: list, start: int, end: int) -> list:
            """
            Dijkstra helper function. From a given list of shortest connections returns full path.
            :param distances: List of dictionaries containing node number ("node") and list of [from which node,total distance] ("val")
            :param start: source
            :param end: target
            :return: reconstructed path
            """
            path = []
            curr = distances[end]  # set the pointer at the last node of the path
            while str(curr["val"][
                          0]) != '':  # from the last node add the values of "val"[0] key that keep track of the shortest path available.
                path.append(curr["node"])
                curr = distances[curr["val"][0]]
            path.append(start)
            path.reverse()
            return path

        unvisited = set()  # set containing all unvisited nodes
        reachable=[x for x in graph if x.count(0)<len(graph)] # temporary fix
        visited = []
        for j in range(len(graph)):
            unvisited.add(j)
        distances = []
        for i in range(len(graph)):
            if i == start:
                distances.append({"node": i, "val": ['', 0]})
            else:
                distances.append({"node": i, "val": ['',
                                                     'inf']})  # every node as a dictionary with "val" being an array [from which node,total distance]
        current = start
        prev=[]
        while unvisited:  # searching process will continue until it checks all the nodes
            if current not in prev:
                prev.append(current)
            neighbors = [i for i in range(len(graph[current])) if (graph[current][i] != 0 and i in unvisited)]
            for node in neighbors: # consider all neighboring nodes
                dist = distances[node]["val"][1]
                new_dist = distances[current]["val"][1] + graph[current][node]  # calculate the distance from the current node to each neighbor
                if dist == 'inf' or new_dist < dist:  # if the new path is shorter, switch
                    distances[node]["val"][1] = new_dist
                    distances[node]["val"][0] = current
            unvisited.remove(current)  # remove current node from unvisited
            if current not in visited:
                visited.append(current)
            valid = [nodes for nodes in distances if nodes["val"][1] != 'inf' and nodes["node"] in unvisited]  # consider all nodes that connect to current node and are not yet visited
            v = [nodes["val"][1] for nodes in distances if nodes["val"][1] != 'inf' and nodes["node"] in unvisited]  # take their path distances
            if not valid:
                current=visited[visited.index(current)-1]
                unvisited.add(current)
            if not valid and len(visited)==len(reachable):
                break

            for nodes in valid:  # from path distances select the one with the shortest distance
                if nodes["val"][1] == min(v):
                    current = nodes["node"]
        path = reconstruct(distances, start, end)
        length = distances[end]["val"][1]
        return path, length

    def bellman_ford(self, graph: list, start: int, end: int):
        """
        Finds the shortest path between two stops using the Bellman-Ford algorithm.

        Args:
        - start (int): Index of the starting stop.
        - end (int): Index of the ending stop.

        Returns:
        - tuple: A tuple containing the shortest path from the starting stop to the ending stop and the total travel time.
        """
        num_stops = len(graph)
        # Initialize distances from the starting stop to all other stops as infinity
        distances = [float('inf')] * num_stops
        distances[start] = 0

        # Update distances by considering each stop and its neighbors
        for _ in range(num_stops - 1):
            for current_stop in range(num_stops):
                for next_stop in range(num_stops):
                    if graph[current_stop][next_stop] != 0 and distances[current_stop] != float('inf'):
                        if distances[current_stop] + graph[current_stop][next_stop] < distances[next_stop]:
                            distances[next_stop] = distances[current_stop] + graph[current_stop][next_stop]

        # Check for negative cycles
        '''for current_stop in range(num_stops):
            for next_stop in range(num_stops):
                if graph[current_stop][next_stop] != 0 and distances[current_stop] + graph[current_stop][next_stop] < distances[next_stop]:
                    print("The transportation network contains a negative cycle")
                    return [], float('inf')'''

        # Constructing the shortest path from end to start
        path = [end]
        current_stop = end
        length = 0
        while current_stop != start:
            for next_stop in range(num_stops):
                if graph[next_stop][current_stop] != 0 and distances[next_stop] == distances[current_stop] - graph[next_stop][current_stop]:
                    path.insert(0, next_stop)
                    length += graph[next_stop][current_stop]
                    current_stop = next_stop
                    break

        return path, length


    @staticmethod
    def a_star(graph: list, start: int, goal: int) -> tuple:
        """
        A* heuristic algorithm for finding the shortest path in a graph.
        :param graph: a graph of nodes in adjacency matrix form
        :param start: start node
        :param goal: goal node
        :return: tuple
        """

        connection = sqlite3.connect("/Users/dominik/Documents/moje/programowanie/Phyton/Jakniedojade/JakNieDojade/mpk.db")
        cursor = connection.cursor()

        def f_value(node: list) -> float:
            """
            A* Helper function, determining the f_value of the given node.
            :param node: node which f_value will be calculated
            :return: f_value of the given node
            """
            g_value = node[2]
            h_value = np.sqrt((node[3] - x_goal) ** 2 + (node[4] - y_goal) ** 2)
            f_value = g_value + h_value
            return f_value

        def reconstruct_path(closed_list: list, current: list) -> tuple:
            """
            A* helper function to reconstruct the path from start to goal.
            :param closed_list: list of all expanded nodes
            :param current: node whose parent we search for in order to reconstruct the path
            :return: tuple - [0] being the reconstructed path and [1] being the total cost of the path
            """

            path = []
            value = current[2]
            while current is not None:
                path.append(current[1])
                parent = current[0]
                current = next((node for node in closed_list if node[1] == parent), None)
            return path[::-1], value

        open_list = []
        closed_list = []
        x_start = cursor.execute(f"select X from Przystanki where IdP = '{start}';").fetchone()[0]
        y_start = cursor.execute(f"select Y from Przystanki where IdP = '{start}';").fetchone()[0]
        # THE WAY NODES IN OPEN AND CLOSED LISTS ARE STORED: [node's parent, node and node's g value, x coordinate, y coordinate]
        open_list.append([None, start, 0, x_start, y_start])
        x_goal = cursor.execute(f"select X from Przystanki where IdP = '{goal}';").fetchone()[0]
        y_goal = cursor.execute(f"select Y from Przystanki where IdP = '{goal}';").fetchone()[0]

        while len(open_list) != 0:  # executing as long as there are neighbours to nodes

            # determining the neighbour in open_list with the lowest f-value
            lowest_f = f_value(open_list[0])
            current = open_list[0]
            for node in open_list:
                if f_value(node) < lowest_f:
                    lowest_f = f_value(node)
                    current = node

            # checking whether current is goal - returning the path
            if current[1] == goal:
                closed_list.append(current)
                return reconstruct_path(closed_list, current)

            open_list.remove(current)
            closed_list.append(current)
            # for each node in the graph
            for i in range(len(graph[0])):
                # if the node is not the neighbour of current or is already in closed list, skip it
                if graph[current[1]][i] == 0 or any(element[1] == i for element in closed_list):
                    continue
                else:
                    # if the neighbour is already in open list
                    if any(element[1] == i for element in open_list):
                        neighbour_g = current[2] + graph[current[1]][i]
                        for j in range(len(open_list)):
                            if open_list[j][1] == i:
                                # If the new g value is lower than the old g value, update the neighbor’s g value and update its parent to the current node
                                if open_list[j][2] > neighbour_g:
                                    x = cursor.execute(f"select X from Przystanki where IdP = '{j}';").fetchone()[0]
                                    y = cursor.execute(f"select Y from Przystanki where IdP = '{j}';").fetchone()[0]
                                    open_list[j] = [current[1], i, neighbour_g, x, y]

                    # if the neighbour is not in the open list, add it to open list
                    else:
                        neighbour_g = current[2] + graph[current[1]][i]
                        x = cursor.execute(f"select X from Przystanki where IdP = '{i}';").fetchone()[0]
                        y = cursor.execute(f"select Y from Przystanki where IdP = '{i}';").fetchone()[0]
                        open_list.append([current[1], i, neighbour_g, x, y])

        # if the open list is empty and path has not been determined: no path possible, return false
        return ([], )

    def timer(self,graph:list,start:int,end:int)->tuple:
        """
        Returns time of searching the shortest path with Dijkstra's algorithm and Bellman-Ford algorithm.
        """
        start_d = tm.perf_counter_ns()
        self.dijkstra(graph,start,end)
        end_d = tm.perf_counter_ns()

        start_bf = tm.perf_counter_ns()
        self.bellman_ford(graph,start,end)
        end_bf = tm.perf_counter_ns()

        return end_d-start_d, end_bf-start_bf

    def match_lines_to_path(self, path: list, lines: list) -> list:
        """
        Matches public transportation lines to a path.
        :param path: a path represented by a list with numbers of nodes in a graph.
        :param lines: a list of public transportation lines represented by dictionaries.
        :returns: returns a list of tuples [(stops, line)]
        """

        def find_matching_subsequence(list1, list2) -> list:
            """
            Helper function for finding a subsequence starting in the first node of list1
            """
            first_element = list1[0]
            list2 = list(dict.fromkeys(list2))  # remove duplicates from the list to prevent bugs

            # Check if there are no subsequences
            try:
                start_index = list2.index(first_element)
            except ValueError:
                return []

            # Check if list1 is a subsequence starting from this index in list2
            if list2[start_index:start_index + len(list1)] == list1:
                return list2[start_index:start_index + len(list1)]

            longest_subsequence = []
            for i in range(len(list1)):
                # Check for the longest subsequence if starting index is in list2
                if start_index + i < len(list2) and list1[i] == list2[start_index + i]:
                    longest_subsequence.append(list1[i])
                else:
                    break

            return longest_subsequence

        route = []
        path_temp = path.copy()

        logging.info(f"Starting match_lines_to_path with path: {path}")

        # Loop over path removing stops
        while len(path_temp) > 0:
            longest_overlap = []
            longest_line = ''
            for line in lines:
                stops = line[0]["Przystanki"]
                overlap = find_matching_subsequence(path_temp, stops)  # Find current overlap using helper function

                # Check for overlaps in reversed line stops
                stops.reverse()
                overlap_reversed = find_matching_subsequence(path_temp, stops)
                stops.reverse()

                # Swap for reversed version if it's longer
                if len(overlap) < len(overlap_reversed):
                    overlap = overlap_reversed

                # Check if current overlap is longer than the previous one, promote trams as they are usually preferred by the people
                if len(overlap) > len(longest_overlap) or (line[0]["Nazwa"] in ['Tramwaj_na_Maslice', 'Tramwaj_na_Swojczyce', 'Tramwaj_Borowska_Szpital', 'Tramwaj_na_Klecine', 'Tramwaj_na_Jagodno', 'Tramwaj_na_Ołtaszyn', 'Tramwaj_na_Gajowice', 'Tramwaj_na_Gądów','Tramwaj_na_Psie_Pole'] and len(overlap)==len(longest_overlap)):
                    longest_overlap = overlap
                    longest_line = line[0]["Nazwa"]

            logging.info(f"Current longest_overlap: {longest_overlap} with line: {longest_line}")

            if longest_overlap:
                name = f"{longest_overlap[0]}-{longest_overlap[-1]}"  # Initialize name for dictionary (it can't store lists as key)
                route.append((name, longest_line))  # Add info to dictionary

                # Remove current longest overlap from path
                if len(longest_overlap) > 2:
                    path_temp = path_temp[
                                len(longest_overlap) - 1:]  # Update path_temp correctly, preserving the last element
                else:
                    path_temp = path_temp[1:]  # Update path_temp correctly

                logging.info(f"Updated path_temp: {path_temp}")
            else:
                break

            # Check if path has 1 element, which means we have finished matching lines, thus remove the last element
            if len(path_temp) == 1:
                path_temp.pop()

        logging.info(f"Final route: {route}")

        return route

In [6]:
class Visualizer:
    def __init__(self):
        project_root = find_project_root()
        self.db = sqlite3.connect(project_root / 'mpk.db')
        self.cursor = self.db.cursor()

    def get_nodes_from_graph(self, graph: list, path = []) -> tuple:
        """
        Converts a graph in adjacency matrix form into a collection of connections between nodes.
        :param graph: a graph in adjacency matrix form.
        :param start: node IDs for showing the shortest path between them on the graph.
        :param end: node IDs for showing the shortest path between them on the graph.
        :returns: returns (all connections, connections in the shortest path between stops given).
        """
        sp = ShortestPath.ShortestPath()
        connections = []
        nodes_in_path = []

        if path:
            for i in range(len(path) - 1):
                nodes_in_path.append((path[i], path[i + 1]))  # add adjacent nodes from the path extracted by the algorithm to the graphs nodes

        else:
            path = []

        for j in range(len(graph)):
            for k in range(len(graph)):
                if j != k:
                    if graph[j][k] != 0:
                        connections.append((j, k))  # append connections between two nodes represented by j,k
        return connections, nodes_in_path

    def get_node_sizes(self, graph: list) -> dict:
        """
        Adjusts node sizes in a graph based on the number of possible destinations from the stop that the node represents.
        :param graph: a graph in adjacency matrix form.
        :returns: returns a dictionary of node sizes corresponding to every node index.
        """
        node_sizes = {}
        for j in range(len(graph)):
            counter = 0  # count the amount of connections from every node
            for k in range(len(graph)):
                if j != k and graph[j][k] != 0:
                    counter += 1
            if counter >= 5:  # for big nodes
                node_sizes[j] = counter * 15
            else:  # for small nodes
                node_sizes[j] = 8  # possible change
        return node_sizes

    def get_vertice_styles(self, networkx_graph: nx.Graph, connections_in_path: list) -> tuple:
        """
        Adjusts styling of networkx graph vertices for path vertices and other vertices.
        :param networkx_graph: a networkx graph class with already existing edges.
        :param connections_in_path: edges belonging to the shortest path between two nodes calculated in get_nodes_from_graph method.
        :returns: a tuple of dictionaries with vertice colors and widths
        """
        vertice_colors = {}
        vertice_width = {}

        for edge in networkx_graph.edges:
            # check if edge is in path
            # networkx doesn't allow duplicates thus also check for revers node IDs in an edge.
            if edge in connections_in_path or (edge[1], edge[0]) in connections_in_path:
                vertice_colors[edge] = 'purple'  # possible changes here
                vertice_width[edge] = 4  # possible changes here
            else:
                vertice_colors[edge] = 'pink'  # possible changes here
                vertice_width[edge] = 2  # possible changes here
        return vertice_colors, vertice_width

    def get_pos(self, networkx_graph: nx.Graph) -> dict:
        """
        For each node gets its coordinates from the database.
        :param networkx_graph: a networkx graph class with already existing nodes.
        :returns: a dictionary with every node's coordinates.
        """
        pos = {}
        for node in networkx_graph.nodes:
            coordinates = self.cursor.execute(f"SELECT Y,X FROM Nowe_przystanki WHERE IdP = '{node}'").fetchone()  # get nodes coordinates from the db
            pos[node] = coordinates
        return pos

    def get_labels(self, networkx_graph: nx.Graph, node_sizes: dict) -> dict:
        """
        For each node determine displayed label.
        :param networkx_graph: a networkx graph class with already existing nodes.
        :param node_sizes: a dictionary where each node ID has size assigned.
        :returns: a dictionary with custom labels for each node.
        """
        labels = {}
        for node in networkx_graph.nodes:
            if node_sizes[node] >= 75:  # check if node has more than 5 connection, terrible design needs changed
                name = self.cursor.execute(f"SELECT Nazwa FROM Nowe_przystanki WHERE IdP = '{node}'").fetchone()  # get stop's name from the db
                labels[node] = name[0]
            else:
                labels[node] = ''  # for small stops do not display any label
        return labels

    def draw_graph(self, graph: list,filename: str, path = []):  # mozna zmienic to zeby bralo start i end losowo, wtedy mozna dowolna trase zaznaczyc
        """
        Draws a graph in an adjacency matrix form.
        """
        connections, connections_in_path = self.get_nodes_from_graph(graph, path)  # possible change for simulation purposes
        node_sizes = self.get_node_sizes(graph)

        G = nx.Graph()
        G.add_edges_from(connections)

        vertice_colors, vertice_width = self.get_vertice_styles(G, connections_in_path)

        pos = self.get_pos(G)
        sizes = [node_sizes[node] for node in G.nodes]
        colors = [vertice_colors[edge] for edge in G.edges()]
        widths = [vertice_width[edge] for edge in G.edges()]
        labels = self.get_labels(G, node_sizes)

        plt.figure(figsize=(20, 20))
        nx.draw_networkx_nodes(G, pos, node_size=sizes, node_color='grey')
        nx.draw_networkx_edges(G, pos, edge_color=colors, width=widths)
        nx.draw_networkx_labels(G, pos, labels=labels, font_size=6, font_color='black', font_weight='bold',
                                verticalalignment='center', horizontalalignment='left',
                                bbox=dict(facecolor='white', alpha=0.4, edgecolor='none',
                                          boxstyle='round,pad=0.2'))
        plt.savefig(filename)
        plt.close()

In [7]:
con = sqlite3.connect('D:\PyCharm\PyCharm 2023.2.4\JakNieDojade\mpk.db')
cursor = con.cursor()

In [8]:
file = open('D:\PyCharm\PyCharm 2023.2.4\JakNieDojade\Dane\graph.json','r')
graph = json.load(file)

In [9]:
gis = GIS(api_key = 'AAPK32cfafb36cf845faabb512edf3385129OaqTvR2s5ASHCY-SAga4AWxlU94IqARqp-qIHkwqpisyy2PSWpkouNvwTO3a134W')
map = gis.map()
map

MapView(layout=Layout(height='400px', width='100%'))

In [21]:
def draw_path(graph,start,end):
    sp = ShortestPath()
    p = sp.dijkstra(graph,start,end)[0]
    print(p)
    path = []
    for stops in p:
        yx = cursor.execute(f"SELECT Y,X FROM Przystanki WHERE Idp = '{stops}'").fetchone()
        path.append([yx[0],yx[1]])
    print(path)
    polyline = Polyline(
        {
            "paths": path
        }
    )

    polyline_attributes = {"name": "Polyline", "description": "I am a Polyline"} # zmienić na nazwy linii, opcjonalnie każdy fragment inną linią zaznaczyć innym kolorem 

    simple_line_symbol = {
        "type": "esriSLS",
        "style": "esriSLSolid",
        "color": [255, 155, 128],
        "width": 2,
    }
    map.draw(
        shape=polyline,
        symbol=simple_line_symbol,
        attributes=polyline_attributes,
        popup={
            "title": polyline_attributes["name"],
            "content": polyline_attributes["description"],
        },
    )


In [11]:
def map_stops():
    con = con = sqlite3.connect('D:\PyCharm\PyCharm 2023.2.4\JakNieDojade\mpk.db')
    cursor = con.cursor()
    yx_all = cursor.execute("SELECT Nazwa,Y,X FROM Przystanki").fetchall()
    for yx in yx_all:
        point = Point({'x':yx[1],'y':yx[2]})
        
        simple_marker_symbol = {
            "type": "esriSMS",
            "style": "esriSMSCircle",
            "color": [0, 0, 0], # zmienic kolory tutaj
            "outline": {"color": [255, 255, 255], "width": 1},
        }
        
        point_attributes = {"name": yx[0], "description": "I am a point"}  # jakis pomysl na description by sie przydal
        
        map.draw(
            shape=point,
            symbol=simple_marker_symbol,
            attributes=point_attributes,
            popup={
                "title": point_attributes["name"],
                "content": point_attributes["description"],
            },
        )
map_stops()

In [24]:
map = gis.map()
map
draw_path(graph,259,300)

[259, 769, 115, 757, 395, 325, 555, 509, 486, 224, 533, 36, 33, 70, 573, 715, 291, 35, 266, 351, 194, 88, 593, 595, 256, 574, 293, 899, 294, 300]
[[17.005902651824226, 51.17959489315847], [16.99955545310727, 51.17837255894701], [16.998301913759743, 51.17596381345519], [16.993293528494245, 51.175628271268224], [16.994674494893893, 51.17334031466762], [16.997121184615278, 51.16913178356313], [16.99858997067332, 51.16607987839261], [17.003291795637253, 51.16228373870148], [17.010376425982304, 51.15748722948624], [17.017640526393667, 51.151021361918744], [17.020508875669904, 51.14801536493131], [17.02626258093067, 51.14166395962282], [17.02998278896388, 51.137405955668434], [17.03664895426128, 51.135844792966054], [17.043933440241048, 51.13680600489449], [17.048344124798884, 51.13727450809283], [17.053687278383467, 51.13793170102513], [17.057197787746514, 51.13521821025446], [17.065250678350605, 51.13205230482232], [17.069754619376962, 51.13603964860488], [17.074723706057206, 51.1380293119

In [25]:
map = gis.map()
map

MapView(layout=Layout(height='400px', width='100%'))

In [26]:
draw_path(graph,259,300)

[259, 769, 115, 757, 395, 325, 555, 509, 486, 224, 533, 36, 33, 70, 573, 715, 291, 35, 266, 351, 194, 88, 593, 595, 256, 574, 293, 899, 294, 300]
[[17.005902651824226, 51.17959489315847], [16.99955545310727, 51.17837255894701], [16.998301913759743, 51.17596381345519], [16.993293528494245, 51.175628271268224], [16.994674494893893, 51.17334031466762], [16.997121184615278, 51.16913178356313], [16.99858997067332, 51.16607987839261], [17.003291795637253, 51.16228373870148], [17.010376425982304, 51.15748722948624], [17.017640526393667, 51.151021361918744], [17.020508875669904, 51.14801536493131], [17.02626258093067, 51.14166395962282], [17.02998278896388, 51.137405955668434], [17.03664895426128, 51.135844792966054], [17.043933440241048, 51.13680600489449], [17.048344124798884, 51.13727450809283], [17.053687278383467, 51.13793170102513], [17.057197787746514, 51.13521821025446], [17.065250678350605, 51.13205230482232], [17.069754619376962, 51.13603964860488], [17.074723706057206, 51.1380293119

In [43]:
map = gis.map()
map


MapView(layout=Layout(height='400px', width='100%'))

In [44]:
draw_path(graph,3,12)


[3, 779, 276, 695, 95, 611, 645, 445, 622, 212, 712, 799, 656, 613, 505, 32, 479, 744, 12]
[[16.963107473334045, 51.073997602922944], [16.963229133477736, 51.077346010396084], [16.9687470693037, 51.07638831205335], [16.97284118962041, 51.076336561127704], [16.97754799383596, 51.07660160743546], [16.98423984686341, 51.07738133758759], [16.99208278388362, 51.079433421722264], [17.001018765928563, 51.081741720709054], [17.006915872857498, 51.08469432093162], [17.012038690891746, 51.08638626845392], [17.01842214304806, 51.08491164842816], [17.02502604649022, 51.083890334818435], [17.030494320378956, 51.08335878151244], [17.03488914330517, 51.083498082602155], [17.04077621896258, 51.08316262649301], [17.049337952214465, 51.083007109561976], [17.053640874293475, 51.082935676158804], [17.059955498906287, 51.08372167539352], [17.065769946395573, 51.08544610886502]]
